In [1]:
import pandas as pd
import numpy  as np
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler #feature transformer that merges multiple columns into a vector column
from pyspark.ml.classification import LogisticRegression
#from pyspark.ml.evaluation import BinaryClassificationEvaluator #evaluate accuracy 
#from pyspark.context import SparkContext
#from pyspark.sql.session import SparkSession
sc= SparkContext(appName="my_app1", master= "local[1]") 
sqlContext = SQLContext(sc) 

In [2]:
""" UseCase2 - This is a demonstration of Spark ML Libraries capabilities 
for which I have tried to implement Logistic Regression Model to predict wicket fall in World Cup Semi Final
 (between India and New Zealand) held in Old Trafford, Manchester, England. """

'''
Steps - 

1.Load the scrapped data of matches and player
2.Prepare semi finals data set [from the scrapped matches data] - -46th Match in Cricket World Cup 2019
3.Prepare Player Bowling Styles Data [from the scrapped players data]
3.Generate Data of wicket falls[FoW] pattern for Team - India in Semi Finals
4.Create wicket falls data for team India in semifinals by merging bowling style data with semifinal data
5.Preprocess data -
  - Add new columns -
      -balls_faced
      -cumruns[total runs a team score on a particular ball]
      -cumrunsbyballsfaced[derived variable calculated from cummulative runs / balls faced]- 
          this value will decrease with more and more unplayed balls during the match
      
6.Implement logistic regression using pyspark MLlib modules to predict wicket falls.

'''


'\nSteps - \n\n1.Load the scrapped data of matches and player\n2.Prepare semi finals data set [from the scrapped matches data] - -46th Match in Cricket World Cup 2019\n3.Prepare Player Bowling Styles Data [from the scrapped players data]\n3.Generate Data of wicket falls[FoW] pattern for Team - India in Semi Finals\n4.Create wicket falls data for team India in semifinals by merging bowling style data with semifinal data\n5.Preprocess data -\n  - Add new columns -\n      -balls_faced\n      -cumruns[total runs a team score on a particular ball]\n      -cumrunsbyballsfaced[derived variable calculated from cummulative runs / balls faced]- \n          this value will decrease with more and more unplayed balls during the match\n      \n6.Implement logistic regression using pyspark MLlib modules to predict wicket falls.\n\n'

In [3]:
'UseCase2.Step1 - Load the scrapped datasets of matches and players'

matches_data = pd.read_csv("C:\\...\\Cricket World Cup 2019 Analysis\\playersdata\\Matches_data.csv")
print("Matches data-")
display(matches_data)
players_data = pd.read_csv("C:\\...\\Cricket World Cup 2019 Analysis\\playersdata\\Players_data.csv")
print("\n\nPlayers data-")
display(players_data)

#unstructured players data

Matches data-


,Match_No,Match_date,Batting_team,Ball_no,Bowler,Batsmen,Runs,commentary
0,1st match,May 30 2019,ENG,NaN,NaN,NaN,NaN,NaN
1,1st match,May 30 2019,ENG,49.6,Rabada,Plunkett,1,"Rabada to Plunkett, 1 run, full again, can onl..."
2,1st match,May 30 2019,ENG,49.5,Rabada,Plunkett,2,"Rabada to Plunkett, 2 runs, fired in towards t..."
3,1st match,May 30 2019,ENG,49.4,Rabada,Archer,1,"Rabada to Archer, 1 run, pitched up and whippe..."
4,1st match,May 30 2019,ENG,49.3,Rabada,Archer,2,"Rabada to Archer, 2 runs, fullish, 86mph and f..."
...,...,...,...,...,...,...,...,...
28746,Final,Jul 14 2019,ENG,0.5,Boult,Bairstow,0,"Boult to Bairstow, no run, looks to flick, bal..."
28747,Final,Jul 14 2019,ENG,0.4,Boult,Bairstow,0,"Boult to Bairstow, no run, back of a length, h..."
28748,Final,Jul 14 2019,ENG,0.3,Boult,Bairstow,0,"Boult to Bairstow, no run, fullish, fourth-stu..."
28749,Final,Jul 14 2019,ENG,0.2,Boult,Roy,1,"Boult to Roy, 1 run, full again, more off-stum..."




Players data-


,Team,Player_Name,Playera_Role,Age,Player_Role,Batting,Bowling
0,Afghanistan Squad / Players,Gulbadin Naib,Captain,Captain,Playing role: Bowler,Batting: Right-hand bat,Bowling: Right-arm medium-fast
1,Afghanistan Squad / Players,Asghar Afghan,NaN,Age: 31 years 117 days,Playing role: Middle-order batsman,Batting: Right-hand bat,Bowling: Right-arm medium-fast
2,Afghanistan Squad / Players,Dawlat Zadran,NaN,Age: 31 years 34 days,Playing role: Bowler,Batting: Right-hand bat,Bowling: Right-arm fast-medium
3,Afghanistan Squad / Players,Hamid Hassan,NaN,Age: 31 years 325 days,Playing role: Bowler,Batting: Right-hand bat,Bowling: Right-arm fast
4,Afghanistan Squad / Players,Hashmatullah Shahidi,NaN,Age: 24 years 169 days,Playing role: Top-order batsman,Batting: Left-hand bat,Bowling: Right-arm offbreak
...,...,...,...,...,...,...,...
185,World Cup 2019 Umpires list / Players,Sundaram Ravi,Umpire,Umpire,NaN,NaN,NaN
186,World Cup 2019 Umpires list / Players,Paul Reiffel,Umpire,Umpire,Playing role: Bowler,Batting: Right-hand bat,Bowling: Right-arm fast-medium
187,World Cup 2019 Umpires list / Players,Rod Tucker,Umpire,Umpire,Batting: Left-hand bat,Bowling: Right-arm medium,NaN
188,World Cup 2019 Umpires list / Players,Joel Wilson,Umpire,Umpire,NaN,NaN,NaN


In [4]:
'UseCase2.Step2 - Prepare semi finals data set [from the scrapped matches data] - -46th Match in Cricket World Cup 2019'
 
testdf1 = matches_data.drop(['commentary'], axis=1)
testdf1 = testdf1.dropna()
testdf1['runs'] = testdf1['Runs'].astype(str).str[0]
testdf1['extras_type'] = testdf1['Runs'].astype(str).str[1:]
testdf1 = testdf1.replace(r'^\s*$', np.nan, regex=True)

#Split by delimiter(.) for Over column and introduce two columns- over and ball  --> 
testdf1[['over','ball']] = testdf1['Ball_no'].astype(str).str.split('.', expand=True)
testdf1['wicket'] = np.where(testdf1['runs'] == "W", 1, 0)
matchdata = testdf1.drop(['Ball_no', 'Runs'], axis=1)
matchdata['over'] = matchdata['over'].astype(np.int64)
matchdata['ball'] = matchdata['ball'].astype(np.int64)
matchdata['wicket'] = matchdata['wicket'].astype(np.int64)
del testdf1

matchdata.runs.replace(['W'], ['0'], inplace= True)
matchdata["runs"] = matchdata["runs"].astype('int64')
matchdata['Match_No'] = matchdata['Match_No'].str[0:5]
matchdata['datetime'] = pd.to_datetime(matchdata['Match_date'], dayfirst = True)
matchdata['month'] = matchdata['datetime'].apply(lambda x: x.strftime('%b')) 
matchdata['date'] = matchdata['datetime'].apply(lambda x: x.strftime('%d'))


#there were total 48 matches and semi-finals listed on 46th number.
matchnumber = {'1st m': 1, '2nd m': 2, '3rd m': 3,'4th m': 4, '5th m': 5, '6th m': 6, '7th m': 7, 
         '8th m': 8, '9th m': 9, '10th ': 10, '12th ': 12, '13th ': 13, '14th ': 14, '15th ': 15,
         '17th ': 17, '19th ': 19, '20th ': 20, '21st ': 21, '22nd ': 22, '23rd ': 23, '24th ': 24,
         '25th ': 25, '26th ': 26, '27th ': 27, '28th ': 28, '29th ': 29, '30th ': 30, '31st ': 31,
         '32nd ': 32, '33rd ': 33, '34th ': 34, '35th ': 35, '36th ': 36, '37th ': 37, '38th ': 38,
         '39th ': 39, '40th ': 40, '41st ': 41, '42nd ': 42, '43rd ': 43, '44th ': 44, '45th ': 45,
         '1st S': 46, '2nd S': 47, 'Final': 48} 
matchdata.Match_No = [matchnumber[i] for i in matchdata.Match_No] 

semifinal = matchdata.sort_values(['Match_No', 'over'], ascending=[True, True])
semifinal = semifinal[semifinal['Match_No'] == 46]
print("Semi Finals Data of IN vs NZ:-")
display(semifinal)

Semi Finals Data of IN vs NZ:-


,Match_No,Match_date,Batting_team,Bowler,Batsmen,runs,extras_type,over,ball,wicket,datetime,month,date
27078,46,Jul 9-10 2019,NZ,Kumar,Guptill,0,NaN,0,6,0,2010-07-09 20:19:00,Jul,09
27079,46,Jul 9-10 2019,NZ,Kumar,Guptill,0,NaN,0,5,0,2010-07-09 20:19:00,Jul,09
27080,46,Jul 9-10 2019,NZ,Kumar,Guptill,0,NaN,0,4,0,2010-07-09 20:19:00,Jul,09
27081,46,Jul 9-10 2019,NZ,Kumar,Guptill,0,NaN,0,3,0,2010-07-09 20:19:00,Jul,09
27082,46,Jul 9-10 2019,NZ,Kumar,Guptill,0,NaN,0,2,0,2010-07-09 20:19:00,Jul,09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26730,46,Jul 9-10 2019,NZ,Bumrah,Boult,0,NaN,49,2,0,2010-07-09 20:19:00,Jul,09
26731,46,Jul 9-10 2019,NZ,Bumrah,Santner,1,NaN,49,1,0,2010-07-09 20:19:00,Jul,09
27084,46,Jul 9-10 2019,INDIA,Neesham,Chahal,0,NaN,49,3,1,2010-07-09 20:19:00,Jul,09
27085,46,Jul 9-10 2019,INDIA,Neesham,Chahal,0,NaN,49,2,0,2010-07-09 20:19:00,Jul,09


In [5]:
'UseCase2.Step3 - Prepare Player Bowling Styles Data [from the scrapped players data]'

#extract New Zealand and India Semifinals data to get bowling style of players in these teams.

fsem_pinfo =players_data[(players_data['Team'] == 'New Zealand Squad / Players') | (players_data['Team'] == 'India Squad / Players')]
fsem_pinfo = fsem_pinfo[['Team', 'Player_Name', 'Bowling']]
fsem_pinfo = fsem_pinfo.dropna()
fsem_pinfo['Player_Name'] = fsem_pinfo.Player_Name.str.strip()
semifinalteams = {'New Zealand Squad / Players': 'NZ', 'India Squad / Players': 'INDIA'} 
fsem_pinfo.Team = [semifinalteams[i] for i in fsem_pinfo.Team] 

# Remove rows with string value as "Notes:"  since this does not describe bowling type -
fsem_pinfo = fsem_pinfo[~fsem_pinfo.Bowling.str.contains("Notes:")]
# Remove string "Bowling:"  since it is common across all rows -
fsem_pinfo['Bowling'] = fsem_pinfo['Bowling'].str.strip('Bowling:')

#Remove whitespaces from the column with object data type -
fsem_pinfo['Bowling'] = fsem_pinfo['Bowling'].str.strip()
fsem_pinfo['Player_Name'] = fsem_pinfo['Player_Name'].str.strip()
fsem_pinfo['Team'] = fsem_pinfo['Team'].str.strip()
fsem_pinfo.columns = ['Batting_team', 'Bowler1', 'BowlingStyle']
fsem_pinfo[['bowler_firstname','bowler_lastname1','bowler_lastname2']] =fsem_pinfo['Bowler1'].astype(str).str.split(' ', expand=True)
fsem_pinfo = fsem_pinfo.replace(np.nan, '', regex=True)
fsem_pinfo['Bowler'] = fsem_pinfo['bowler_lastname1'].astype(str) + " " +  fsem_pinfo['bowler_lastname2']
fsem_pinfo = fsem_pinfo[['Bowler', 'BowlingStyle']]
fsem_pinfo['Bowler'] = fsem_pinfo.Bowler.str.strip()
fsem_pinfo['BowlingStyle'] = fsem_pinfo.BowlingStyle.str.strip()
print("Bowling style of players-")
display(fsem_pinfo)


Bowling style of players-


,Bowler,BowlingStyle
71,Kohli,Right-arm medium
72,Sharma,Right-arm offbreak
74,Bumrah,Right-arm fast
75,Chahal,Legbreak googly
76,Dhoni,Right-arm medium
77,Jadeja,Slow left-arm orthodox
78,Jadhav,Right-arm offbreak
80,Yadav,Slow left-arm chinama
81,Kumar,Right-arm medium
82,Shami,Right-arm fast


In [6]:
" UseCase2.Step4 - Create wicket falls data for team India in semifinals by merging bowling style data with semifinal data"

#Batting team - INDIA - wicket fall data 
semifinal_BT1 = semifinal[semifinal['Batting_team'] == 'INDIA']
wicketfalldata = pd.merge(fsem_pinfo, semifinal_BT1[['Bowler', 'over', 'ball', 'wicket', 'runs']], on='Bowler')
wicketfalldata_IND  = wicketfalldata.sort_values(['over', 'ball'], ascending=[True, True])
display(wicketfalldata_IND) # this contains overwise details of bowler with their wickets taken and runs conceded.
#wicketfalldata_IND.to_csv("wicketfalldata_IND.csv")

print("Fall of Wicket pattern data(top 20 obs.) - Team India-")
wktfall_data= sqlContext.createDataFrame(wicketfalldata_IND)
wktfall_data.createOrReplaceTempView("wicketfall_IND")
display(wktfall_data.show(truncate=False))
wktfall_data.printSchema()

,Bowler,BowlingStyle,over,ball,wicket,runs
5,Boult,Left-arm fast-medium,0,1,0,1
4,Boult,Left-arm fast-medium,0,2,0,1
3,Boult,Left-arm fast-medium,0,3,0,0
2,Boult,Left-arm fast-medium,0,4,0,0
1,Boult,Left-arm fast-medium,0,5,0,0
...,...,...,...,...,...,...
133,Ferguson,Right-arm fast,48,5,0,1
132,Ferguson,Right-arm fast,48,6,1,0
248,Neesham,Right-arm medium-fast,49,1,0,4
247,Neesham,Right-arm medium-fast,49,2,0,0


Fall of Wicket pattern data(top 20 obs.) - Team India-
+------+---------------------+----+----+------+----+
|Bowler|BowlingStyle         |over|ball|wicket|runs|
+------+---------------------+----+----+------+----+
|Boult |Left-arm fast-medium |0   |1   |0     |1   |
|Boult |Left-arm fast-medium |0   |2   |0     |1   |
|Boult |Left-arm fast-medium |0   |3   |0     |0   |
|Boult |Left-arm fast-medium |0   |4   |0     |0   |
|Boult |Left-arm fast-medium |0   |5   |0     |0   |
|Boult |Left-arm fast-medium |0   |6   |0     |0   |
|Henry |Right-arm fast-medium|1   |1   |0     |0   |
|Henry |Right-arm fast-medium|1   |2   |0     |2   |
|Henry |Right-arm fast-medium|1   |3   |1     |0   |
|Henry |Right-arm fast-medium|1   |4   |0     |0   |
|Henry |Right-arm fast-medium|1   |5   |0     |1   |
|Henry |Right-arm fast-medium|1   |6   |0     |0   |
|Boult |Left-arm fast-medium |2   |1   |0     |0   |
|Boult |Left-arm fast-medium |2   |2   |0     |0   |
|Boult |Left-arm fast-medium |2   |3   |0   

None

root
 |-- Bowler: string (nullable = true)
 |-- BowlingStyle: string (nullable = true)
 |-- over: long (nullable = true)
 |-- ball: long (nullable = true)
 |-- wicket: long (nullable = true)
 |-- runs: long (nullable = true)



In [7]:
'UseCase2.Step5-Preprocess data'

#Create Dummies for Bowling Type -
bowlingstyle_categories = wktfall_data.select("BowlingStyle").distinct().rdd.flatMap(lambda x: x).collect()

bowlingstyle_binary_dummies = [F.when(F.col("BowlingStyle") == cat, 1).otherwise(0).alias(cat)
         for cat in bowlingstyle_categories]

wktfall_data1 = wktfall_data.select("BowlingStyle", "over", "ball", "wicket", "runs", *bowlingstyle_binary_dummies)
display(wktfall_data1.show(truncate=False))
wktfall_data1.printSchema()

#create three new columns- balls_faced, cumruns[total runs a team score on a particular ball], cumrunperball[derived variable calculated from cummulative runs / balls faced].

#Convert to pandas dataframe for ease in computations
wktfall_data2 = wktfall_data1.toPandas()
#Create a new column of balls faced - 
wktfall_data2['balls_faced'] = range(1, wktfall_data2.shape[0]+1)
#Create a new columns as cummruns i.e.total team score on a particular ball  -
wktfall_data2['cumruns'] = wktfall_data2['runs'].cumsum(axis = 0)
wktfall_data2['cumrunsbyballsfaced'] = wktfall_data2['cumruns']/wktfall_data2['balls_faced']
#this will decrease with more and more unplayed balls during the match

#subset the above data to obtain the needed information/features.
wktfall_data3 = wktfall_data2[['cumrunsbyballsfaced', 'Slow left-arm orthodox', 'Right-arm fast', 'Right-arm medium-fast', 'Right-arm fast-medium', 'Left-arm fast-medium', 'wicket']]
wktfall_data3.head()
wktfall_data4 = sqlContext.createDataFrame(wktfall_data3)
print("\nPreprocessed wicket falls data-")
wktfall_data4.show(truncate=False)


#Use Spark ML libraries to generate input features for logistic reg model
in_features = wktfall_data4.columns[:-1]
assembler = VectorAssembler(inputCols = in_features,outputCol="features") 
##Vector Assembler is a feature transformer that merges multiple columns into a vector column
wktfall_pred = assembler.transform(wktfall_data4)
wktfall_pred.show(truncate=False)


#Divide the wicket fall data in traindata and testdata for building model on traindata
traindata, testdata = wktfall_pred.randomSplit([0.7, 0.3], seed = 7)
print("\nObservations from Train data set:")
traindata.show(truncate=False)
print("\nObservations from Test data set:")
testdata.show(truncate=False)

+---------------------+----+----+------+----+----------------------+--------------+---------------------+---------------------+--------------------+
|BowlingStyle         |over|ball|wicket|runs|Slow left-arm orthodox|Right-arm fast|Right-arm medium-fast|Right-arm fast-medium|Left-arm fast-medium|
+---------------------+----+----+------+----+----------------------+--------------+---------------------+---------------------+--------------------+
|Left-arm fast-medium |0   |1   |0     |1   |0                     |0             |0                    |0                    |1                   |
|Left-arm fast-medium |0   |2   |0     |1   |0                     |0             |0                    |0                    |1                   |
|Left-arm fast-medium |0   |3   |0     |0   |0                     |0             |0                    |0                    |1                   |
|Left-arm fast-medium |0   |4   |0     |0   |0                     |0             |0                    |0

None

root
 |-- BowlingStyle: string (nullable = true)
 |-- over: long (nullable = true)
 |-- ball: long (nullable = true)
 |-- wicket: long (nullable = true)
 |-- runs: long (nullable = true)
 |-- Slow left-arm orthodox: integer (nullable = false)
 |-- Right-arm fast: integer (nullable = false)
 |-- Right-arm medium-fast: integer (nullable = false)
 |-- Right-arm fast-medium: integer (nullable = false)
 |-- Left-arm fast-medium: integer (nullable = false)


Preprocessed wicket falls data-
+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+
|cumrunsbyballsfaced|Slow left-arm orthodox|Right-arm fast|Right-arm medium-fast|Right-arm fast-medium|Left-arm fast-medium|wicket|
+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+
|1.0                |0                     |0             |0                    |0                    |1           

+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+-----------------------------------+
|cumrunsbyballsfaced|Slow left-arm orthodox|Right-arm fast|Right-arm medium-fast|Right-arm fast-medium|Left-arm fast-medium|wicket|features                           |
+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+-----------------------------------+
|0.1875             |0                     |0             |0                    |1                    |0                   |0     |(6,[0,4],[0.1875,1.0])             |
|0.20689655172413793|0                     |0             |0                    |0                    |1                   |0     |(6,[0,5],[0.20689655172413793,1.0])|
|0.21428571428571427|0                     |0             |0                    |0                    |1                   |0     |(6,[0,5],[0.21428571428571427

In [8]:
'UseCase2.Step6-Implement logistic regression using pyspark MLlib modules to predict wicket falls.'

#Apply logistic reg function for implementing logistic reg model
logmodel = LogisticRegression(featuresCol="features", labelCol="wicket")

# Fit the logistic regression model - 
model=logmodel.fit(traindata)

# Get the coefficients and intercept for logistic regression
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))


trainingSummary = model.summary 
accuracy = trainingSummary.accuracy
print("Fitted Model accuracy on Train Set: " + str(accuracy))



#Predict on test data -
print("\nPredictions on test data set-")
predict_test=model.transform(testdata)
predict_test.show(truncate=False)


test_acc = predict_test.select("wicket", "prediction")

test_acc_arr = np.array((test_acc.collect()))
test_acc_arr
from sklearn.metrics import accuracy_score,precision_score, recall_score, roc_auc_score, f1_score
acc = accuracy_score(test_acc_arr[:,0], test_acc_arr[:,1])
f1 = f1_score(test_acc_arr[:,0], test_acc_arr[:,1])
precision = precision_score(test_acc_arr[:,0], test_acc_arr[:,1])
recall = recall_score(test_acc_arr[:,0], test_acc_arr[:,1])
#auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print("Test Set metrics:\n accuracy: " + str(acc) + "\n precision:" + str(precision) 
     + "\n recall:" + str(recall) + "\n f1 score:" + str(f1))

""" Summary -
-The data ingested in our model is highly imbalanced.
-Recall and Precision are zero which implies that there are no true positives in test set. 
-We need to first improve the imbalancing and tune model for improvements.
"""

Coefficients: [1.3974923141131208,-0.4768580328055,-0.3803401638162682,-1.0543037612417197,0.09289639017145575,-0.3770530370185253]
Intercept: -3.349182963318684
Fitted Model accuracy on Train Set: 0.9528301886792453

Predictions on test data set-
+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+-----------------------------------+----------------------------------------+-----------------------------------------+----------+
|cumrunsbyballsfaced|Slow left-arm orthodox|Right-arm fast|Right-arm medium-fast|Right-arm fast-medium|Left-arm fast-medium|wicket|features                           |rawPrediction                           |probability                              |prediction|
+-------------------+----------------------+--------------+---------------------+---------------------+--------------------+------+-----------------------------------+----------------------------------------+--------------------

c:\users\ag89382\appdata\local\programs\python\python37\deepenv\lib\site-packages\sklearn\metrics\_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
c:\users\ag89382\appdata\local\programs\python\python37\deepenv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ag89382\appdata\local\programs\python\python37\deepenv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resul

' Summary -\n-The data ingested in our model is highly imbalanced.\n-Recall and Precision are zero which implies that there are no true positives in test set. \n-We need to first improve the imbalancing and tune model for improvements.\n'

In [9]:
#References:
#https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html